<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Universal-Sentence-Encoder" data-toc-modified-id="Universal-Sentence-Encoder-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Universal Sentence Encoder</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Label-PowerSet" data-toc-modified-id="Label-PowerSet-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Label PowerSet</a></span><ul class="toc-item"><li><span><a href="#Stochastic-Gradient-Descent-Optimizer" data-toc-modified-id="Stochastic-Gradient-Descent-Optimizer-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Stochastic Gradient Descent Optimizer</a></span></li><li><span><a href="#Adam-Optimizer" data-toc-modified-id="Adam-Optimizer-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Adam Optimizer</a></span></li><li><span><a href="#RMSProp-Optimizer" data-toc-modified-id="RMSProp-Optimizer-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>RMSProp Optimizer</a></span></li><li><span><a href="#Adagrad" data-toc-modified-id="Adagrad-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Adagrad</a></span></li><li><span><a href="#Adadelta" data-toc-modified-id="Adadelta-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Adadelta</a></span></li></ul></li><li><span><a href="#Binary-Relevance" data-toc-modified-id="Binary-Relevance-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Binary Relevance</a></span></li></ul></li></ul></div>

# Word Embedding + Neural Network

## Universal Sentence Encoder
USE(Universal Sentence Encoder)는 어떤 문장이든 512차원의 벡터로 바꿔주는 Google에서 제공하는 tool입니다. 
현재 구글이 제공하는 Sentence Encoder 모델이 크게 2개로 분류 되어지는데,

하나는 Transformer Architecture에 기반하며 다른 하나는 Deep Averaging Network(DAN)에 기반합니다.
두 모델 모두 단어 / 문장 / 문단에 대하여 512차원의 벡터로 변환시킵니다.

- Transformer 모델은 정확도에 집중하지만, 메모리를 많이 차지하며 속도도 느립니다
- DAN 모델은 속도와 효율성에 집중하지만 정확도가 비교적 낮을 수 있습니다.

In [13]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import glob
import functools 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import missingno as msno
import re
import os.path
import math
from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from scipy import sparse
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
#from skmultilearn.problem_transform import LabelPowerset
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=0.8)

from helper_functions import *
rdm_seed = 29
np.random.seed(rdm_seed)
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
import keras.optimizers

## Modeling
원본 데이터 및 벡터화된 데이터 불러오기

In [14]:
mydata_train = pd.read_csv('./Data/Prepossed/Prepossed_data_train.csv')
mydata_test = pd.read_csv('./Data/Prepossed/Prepossed_data_test.csv')
mydata =  pd.read_csv('./Data/Prepossed/Prepossed_data.csv')
mydata = mydata.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis = 1)
mydata_train = mydata_train.drop([ 'Unnamed: 0.1'],axis = 1)
mydata_test = mydata_test.drop([ 'Unnamed: 0.1'],axis = 1)

train_X, train_y = mydata_train['Plot'], mydata_train.drop(['BID', 'Plot'], axis=1)
test_X, test_y = mydata_test['Plot'], mydata_test.drop(['BID', 'Plot'], axis=1)

category_columns = train_y.columns

In [15]:
train_vector = np.load("./Data/np/train_Plot.npy", allow_pickle=True)
test_vector = np.load("./Data/np/test_Plot.npy", allow_pickle=True)

### Label PowerSet

신경망을 구축하여 총 540가지의 가능성 중 하나를 예측

신경망 모델링
- Input Layer : 512 (벡터 = 512차원)
- Output Layer : 540 (총 540가지 Unique한 조합)
- Hidden Layer : 525 
- Softmax 사용, Overfitting을 방지하기 위한 20% Dropout

In [22]:
train_y_labels= train_y.groupby(list(category_columns)).ngroup()
y_labels_lut = train_y.copy(deep=True) 
y_labels_lut['Labels'] = train_y_labels
y_labels_lut = y_labels_lut.drop_duplicates()
y_labels_lut = y_labels_lut.reset_index(drop=True).set_index('Labels').sort_index()

# One-hot encoding the output labels
num_classes = y_labels_lut.shape[0]
train_y_onehot = np_utils.to_categorical(train_y_labels, num_classes = num_classes)


#### Stochastic Gradient Descent Optimizer

In [20]:
import keras.backend.tensorflow_backend as K
def gen_model(optimizer):
    with K.tf.device('/device:GPU:0'):
        model = Sequential()
        model.add(Dense(525, activation='relu', input_shape=(512,)))
        model.add(Dropout(0.5))
        model.add(Dense(540, activation='softmax'))
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

lr_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

In [21]:
epochs, batch_size = 600, 128

with K.tf.device('/device:GPU:0'):
    model = gen_model(keras.optimizers.SGD(lr=1))
    model.fit(train_vector, train_y_onehot,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.3,
              callbacks=[lr_reduction])

Train on 2090 samples, validate on 897 samples
Epoch 1/600
2090/2090 [==============================] - 0s 129us/step - loss: 6.0334 - accuracy: 0.0713 - val_loss: 5.5841 - val_accuracy: 0.0825
Epoch 2/600
2090/2090 [==============================] - 0s 206us/step - loss: 5.3696 - accuracy: 0.0679 - val_loss: 5.3108 - val_accuracy: 0.0479
Epoch 3/600
2090/2090 [==============================] - 1s 279us/step - loss: 5.1960 - accuracy: 0.0708 - val_loss: 5.3054 - val_accuracy: 0.0234
Epoch 4/600
2090/2090 [==============================] - 1s 343us/step - loss: 5.1488 - accuracy: 0.0684 - val_loss: 5.3294 - val_accuracy: 0.0702
Epoch 5/600
2090/2090 [==============================] - 1s 427us/step - loss: 5.1071 - accuracy: 0.0737 - val_loss: 5.2478 - val_accuracy: 0.0825
Epoch 6/600
2090/2090 [==============================] - 0s 110us/step - loss: 5.0806 - accuracy: 0.0775 - val_loss: 5.2358 - val_accuracy: 0.0914
Epoch 7/600
2090/2090 [==============================] - 0s 185us/step 

2090/2090 [==============================] - 0s 206us/step - loss: 1.5977 - accuracy: 0.5943 - val_loss: 6.0033 - val_accuracy: 0.0858
Epoch 112/600
2090/2090 [==============================] - 1s 422us/step - loss: 1.5404 - accuracy: 0.6115 - val_loss: 6.0333 - val_accuracy: 0.0858
Epoch 113/600
2090/2090 [==============================] - 1s 288us/step - loss: 1.5325 - accuracy: 0.6067 - val_loss: 6.0869 - val_accuracy: 0.0858
Epoch 114/600
2090/2090 [==============================] - 1s 337us/step - loss: 1.4858 - accuracy: 0.6278 - val_loss: 5.9789 - val_accuracy: 0.1003
Epoch 115/600
2090/2090 [==============================] - 0s 190us/step - loss: 1.4667 - accuracy: 0.6220 - val_loss: 6.0124 - val_accuracy: 0.0780
Epoch 116/600
2090/2090 [==============================] - 1s 341us/step - loss: 1.4148 - accuracy: 0.6507 - val_loss: 6.0166 - val_accuracy: 0.0914
Epoch 117/600
2090/2090 [==============================] - 1s 334us/step - loss: 1.3927 - accuracy: 0.6598 - val_loss: 6

2090/2090 [==============================] - 1s 279us/step - loss: 0.2863 - accuracy: 0.9493 - val_loss: 7.3490 - val_accuracy: 0.1037
Epoch 222/600
2090/2090 [==============================] - 1s 335us/step - loss: 0.2747 - accuracy: 0.9565 - val_loss: 7.1996 - val_accuracy: 0.0881
Epoch 223/600
2090/2090 [==============================] - 1s 290us/step - loss: 0.2702 - accuracy: 0.9569 - val_loss: 7.2485 - val_accuracy: 0.0858
Epoch 224/600
2090/2090 [==============================] - 1s 240us/step - loss: 0.2702 - accuracy: 0.9560 - val_loss: 7.2329 - val_accuracy: 0.0914
Epoch 225/600
2090/2090 [==============================] - 1s 247us/step - loss: 0.2578 - accuracy: 0.9526 - val_loss: 7.2884 - val_accuracy: 0.1003
Epoch 226/600
2090/2090 [==============================] - 1s 340us/step - loss: 0.2653 - accuracy: 0.9493 - val_loss: 7.2923 - val_accuracy: 0.0836
Epoch 227/600
2090/2090 [==============================] - 1s 423us/step - loss: 0.2683 - accuracy: 0.9560 - val_loss: 7

2090/2090 [==============================] - 1s 385us/step - loss: 0.1600 - accuracy: 0.9751 - val_loss: 7.5849 - val_accuracy: 0.0814
Epoch 277/600
2090/2090 [==============================] - 1s 389us/step - loss: 0.1639 - accuracy: 0.9722 - val_loss: 7.6262 - val_accuracy: 0.0892
Epoch 278/600
2090/2090 [==============================] - 0s 205us/step - loss: 0.1663 - accuracy: 0.9766 - val_loss: 7.6127 - val_accuracy: 0.0792
Epoch 279/600
2090/2090 [==============================] - 1s 271us/step - loss: 0.1690 - accuracy: 0.9742 - val_loss: 7.5702 - val_accuracy: 0.0858
Epoch 280/600
2090/2090 [==============================] - 1s 288us/step - loss: 0.1702 - accuracy: 0.9718 - val_loss: 7.5636 - val_accuracy: 0.0825
Epoch 281/600
2090/2090 [==============================] - 1s 385us/step - loss: 0.1663 - accuracy: 0.9746 - val_loss: 7.5728 - val_accuracy: 0.0780
Epoch 282/600
2090/2090 [==============================] - 1s 482us/step - loss: 0.1546 - accuracy: 0.9761 - val_loss: 7

2090/2090 [==============================] - 0s 224us/step - loss: 0.0889 - accuracy: 0.9847 - val_loss: 8.0419 - val_accuracy: 0.0792
Epoch 387/600
2090/2090 [==============================] - 1s 385us/step - loss: 0.1052 - accuracy: 0.9818 - val_loss: 7.9899 - val_accuracy: 0.0814
Epoch 388/600
2090/2090 [==============================] - 1s 248us/step - loss: 0.0929 - accuracy: 0.9809 - val_loss: 8.0152 - val_accuracy: 0.0769
Epoch 389/600
2090/2090 [==============================] - 1s 375us/step - loss: 0.0924 - accuracy: 0.9861 - val_loss: 8.0368 - val_accuracy: 0.0847
Epoch 390/600
2090/2090 [==============================] - 1s 246us/step - loss: 0.0998 - accuracy: 0.9818 - val_loss: 8.0051 - val_accuracy: 0.0825
Epoch 391/600
2090/2090 [==============================] - 0s 204us/step - loss: 0.0906 - accuracy: 0.9880 - val_loss: 8.0369 - val_accuracy: 0.0825
Epoch 392/600
2090/2090 [==============================] - 1s 518us/step - loss: 0.0884 - accuracy: 0.9890 - val_loss: 8

2090/2090 [==============================] - 1s 251us/step - loss: 0.0731 - accuracy: 0.9880 - val_loss: 8.2140 - val_accuracy: 0.0814
Epoch 442/600
2090/2090 [==============================] - 1s 471us/step - loss: 0.0828 - accuracy: 0.9852 - val_loss: 8.1279 - val_accuracy: 0.0792
Epoch 443/600
2090/2090 [==============================] - 1s 434us/step - loss: 0.0704 - accuracy: 0.9909 - val_loss: 8.1554 - val_accuracy: 0.0803
Epoch 444/600
2090/2090 [==============================] - 1s 439us/step - loss: 0.0718 - accuracy: 0.9852 - val_loss: 8.1547 - val_accuracy: 0.0769
Epoch 445/600
2090/2090 [==============================] - 1s 334us/step - loss: 0.0887 - accuracy: 0.9823 - val_loss: 8.1392 - val_accuracy: 0.0769
Epoch 446/600
2090/2090 [==============================] - 1s 382us/step - loss: 0.0864 - accuracy: 0.9789 - val_loss: 8.1458 - val_accuracy: 0.0814
Epoch 447/600
2090/2090 [==============================] - 1s 385us/step - loss: 0.0764 - accuracy: 0.9852 - val_loss: 8

2090/2090 [==============================] - 0s 208us/step - loss: 0.0543 - accuracy: 0.9900 - val_loss: 8.4099 - val_accuracy: 0.0769
Epoch 552/600
2090/2090 [==============================] - 1s 324us/step - loss: 0.0578 - accuracy: 0.9885 - val_loss: 8.3807 - val_accuracy: 0.0792
Epoch 553/600
2090/2090 [==============================] - 1s 241us/step - loss: 0.0659 - accuracy: 0.9847 - val_loss: 8.3809 - val_accuracy: 0.0780
Epoch 554/600
2090/2090 [==============================] - 1s 244us/step - loss: 0.0584 - accuracy: 0.9885 - val_loss: 8.4156 - val_accuracy: 0.0814
Epoch 555/600
2090/2090 [==============================] - 1s 330us/step - loss: 0.0539 - accuracy: 0.9914 - val_loss: 8.3724 - val_accuracy: 0.0803
Epoch 556/600
2090/2090 [==============================] - 0s 193us/step - loss: 0.0565 - accuracy: 0.9904 - val_loss: 8.3605 - val_accuracy: 0.0847
Epoch 557/600
2090/2090 [==============================] - 0s 195us/step - loss: 0.0586 - accuracy: 0.9876 - val_loss: 8

In [23]:
y_pred = model.predict(test_vector)
y_pred_label = pd.DataFrame(np.argmax(y_pred, axis=1))
predictions = pd.merge(y_pred_label, y_labels_lut, how='left', left_on=0, right_on='Labels')[category_columns]
accuracy(test_y, predictions)

,Precision,Recall,F1-Score,Support
math,0.49,0.47,0.48,251.0
dp,0.36,0.37,0.37,193.0
implementation,0.42,0.35,0.38,267.0
greedy,0.42,0.31,0.36,248.0
data structures,0.31,0.28,0.30,158.0
sortings,0.20,0.10,0.14,96.0
dfs,0.28,0.22,0.24,93.0
graphs,0.45,0.40,0.42,104.0
trees,0.62,0.51,0.56,92.0
constructive algorithms,0.29,0.20,0.23,133.0


#### Adam Optimizer

In [24]:
epochs, batch_size = 600, 128
with K.tf.device('/device:GPU:0'):
    model = gen_model(keras.optimizers.Adam(lr=0.001))
    model.fit(train_vector, train_y_onehot,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.3,
              callbacks=[lr_reduction])

Train on 2090 samples, validate on 897 samples
Epoch 1/600
2090/2090 [==============================] - 0s 107us/step - loss: 6.1572 - accuracy: 0.0550 - val_loss: 5.8910 - val_accuracy: 0.0825
Epoch 2/600
2090/2090 [==============================] - 0s 235us/step - loss: 5.4251 - accuracy: 0.0818 - val_loss: 5.3078 - val_accuracy: 0.0825
Epoch 3/600
2090/2090 [==============================] - 1s 381us/step - loss: 5.0839 - accuracy: 0.0799 - val_loss: 5.2920 - val_accuracy: 0.0847
Epoch 4/600
2090/2090 [==============================] - 1s 289us/step - loss: 4.9631 - accuracy: 0.0957 - val_loss: 5.2661 - val_accuracy: 0.0914
Epoch 5/600
2090/2090 [==============================] - 1s 394us/step - loss: 4.8740 - accuracy: 0.0957 - val_loss: 5.2522 - val_accuracy: 0.0936
Epoch 6/600
2090/2090 [==============================] - 0s 235us/step - loss: 4.7800 - accuracy: 0.1072 - val_loss: 5.2266 - val_accuracy: 0.0959
Epoch 7/600
2090/2090 [==============================] - 1s 384us/step 

2090/2090 [==============================] - 0s 100us/step - loss: 0.4595 - accuracy: 0.9072 - val_loss: 7.4074 - val_accuracy: 0.1026
Epoch 112/600
2090/2090 [==============================] - 0s 137us/step - loss: 0.4609 - accuracy: 0.9124 - val_loss: 7.4239 - val_accuracy: 0.1014
Epoch 113/600
2090/2090 [==============================] - 0s 230us/step - loss: 0.4493 - accuracy: 0.9153 - val_loss: 7.4073 - val_accuracy: 0.1003
Epoch 114/600
2090/2090 [==============================] - 1s 381us/step - loss: 0.4453 - accuracy: 0.9100 - val_loss: 7.4646 - val_accuracy: 0.0992
Epoch 115/600
2090/2090 [==============================] - 1s 299us/step - loss: 0.4389 - accuracy: 0.9086 - val_loss: 7.4599 - val_accuracy: 0.1014
Epoch 116/600
2090/2090 [==============================] - 1s 569us/step - loss: 0.4468 - accuracy: 0.9038 - val_loss: 7.4680 - val_accuracy: 0.1003
Epoch 117/600
2090/2090 [==============================] - 1s 289us/step - loss: 0.4450 - accuracy: 0.9053 - val_loss: 7

2090/2090 [==============================] - 0s 196us/step - loss: 0.1907 - accuracy: 0.9708 - val_loss: 8.1219 - val_accuracy: 0.1026
Epoch 167/600
2090/2090 [==============================] - 1s 285us/step - loss: 0.2040 - accuracy: 0.9608 - val_loss: 8.1129 - val_accuracy: 0.1003
Epoch 168/600
2090/2090 [==============================] - 1s 389us/step - loss: 0.1968 - accuracy: 0.9670 - val_loss: 8.1429 - val_accuracy: 0.1037
Epoch 169/600
2090/2090 [==============================] - 0s 187us/step - loss: 0.2070 - accuracy: 0.9651 - val_loss: 8.1355 - val_accuracy: 0.1037
Epoch 170/600
2090/2090 [==============================] - 1s 340us/step - loss: 0.1930 - accuracy: 0.9641 - val_loss: 8.1710 - val_accuracy: 0.1037
Epoch 171/600
2090/2090 [==============================] - 1s 383us/step - loss: 0.1977 - accuracy: 0.9675 - val_loss: 8.1869 - val_accuracy: 0.1014
Epoch 172/600
2090/2090 [==============================] - 0s 82us/step - loss: 0.1979 - accuracy: 0.9665 - val_loss: 8.

2090/2090 [==============================] - 1s 289us/step - loss: 0.1146 - accuracy: 0.9813 - val_loss: 8.6490 - val_accuracy: 0.1014
Epoch 222/600
2090/2090 [==============================] - 1s 241us/step - loss: 0.1065 - accuracy: 0.9842 - val_loss: 8.6443 - val_accuracy: 0.0981
Epoch 223/600
2090/2090 [==============================] - 1s 383us/step - loss: 0.1143 - accuracy: 0.9770 - val_loss: 8.6687 - val_accuracy: 0.1026
Epoch 224/600
2090/2090 [==============================] - 1s 292us/step - loss: 0.1136 - accuracy: 0.9761 - val_loss: 8.7024 - val_accuracy: 0.0981
Epoch 225/600
2090/2090 [==============================] - 1s 387us/step - loss: 0.1050 - accuracy: 0.9866 - val_loss: 8.7269 - val_accuracy: 0.1048
Epoch 226/600
2090/2090 [==============================] - 0s 55us/step - loss: 0.1054 - accuracy: 0.9842 - val_loss: 8.7188 - val_accuracy: 0.1003
Epoch 227/600
2090/2090 [==============================] - 0s 176us/step - loss: 0.1060 - accuracy: 0.9828 - val_loss: 8.

2090/2090 [==============================] - 1s 578us/step - loss: 0.0441 - accuracy: 0.9909 - val_loss: 9.5457 - val_accuracy: 0.1003
Epoch 332/600
2090/2090 [==============================] - 1s 530us/step - loss: 0.0414 - accuracy: 0.9923 - val_loss: 9.5808 - val_accuracy: 0.0992
Epoch 333/600
2090/2090 [==============================] - 1s 287us/step - loss: 0.0473 - accuracy: 0.9885 - val_loss: 9.5312 - val_accuracy: 0.0959
Epoch 334/600
2090/2090 [==============================] - 1s 297us/step - loss: 0.0493 - accuracy: 0.9880 - val_loss: 9.5769 - val_accuracy: 0.0959
Epoch 335/600
2090/2090 [==============================] - 1s 424us/step - loss: 0.0437 - accuracy: 0.9928 - val_loss: 9.5858 - val_accuracy: 0.0981
Epoch 336/600
2090/2090 [==============================] - 1s 390us/step - loss: 0.0430 - accuracy: 0.9900 - val_loss: 9.5803 - val_accuracy: 0.0992
Epoch 337/600
2090/2090 [==============================] - 1s 477us/step - loss: 0.0474 - accuracy: 0.9885 - val_loss: 9

2090/2090 [==============================] - 1s 482us/step - loss: 0.0337 - accuracy: 0.9909 - val_loss: 9.9138 - val_accuracy: 0.1026
Epoch 387/600
2090/2090 [==============================] - 1s 340us/step - loss: 0.0340 - accuracy: 0.9919 - val_loss: 9.8936 - val_accuracy: 0.1003
Epoch 388/600
2090/2090 [==============================] - 1s 433us/step - loss: 0.0390 - accuracy: 0.9880 - val_loss: 9.8851 - val_accuracy: 0.1003
Epoch 389/600
2090/2090 [==============================] - 1s 433us/step - loss: 0.0371 - accuracy: 0.9895 - val_loss: 9.8582 - val_accuracy: 0.0970
Epoch 390/600
2090/2090 [==============================] - 1s 383us/step - loss: 0.0305 - accuracy: 0.9914 - val_loss: 9.8855 - val_accuracy: 0.0925
Epoch 391/600
2090/2090 [==============================] - 1s 440us/step - loss: 0.0347 - accuracy: 0.9923 - val_loss: 9.9503 - val_accuracy: 0.0970
Epoch 392/600
2090/2090 [==============================] - 1s 428us/step - loss: 0.0337 - accuracy: 0.9909 - val_loss: 9

Epoch 441/600
2090/2090 [==============================] - 0s 204us/step - loss: 0.0275 - accuracy: 0.9914 - val_loss: 10.1808 - val_accuracy: 0.0914
Epoch 442/600
2090/2090 [==============================] - 1s 367us/step - loss: 0.0298 - accuracy: 0.9904 - val_loss: 10.2157 - val_accuracy: 0.0936
Epoch 443/600
2090/2090 [==============================] - 1s 295us/step - loss: 0.0220 - accuracy: 0.9957 - val_loss: 10.2853 - val_accuracy: 0.0992
Epoch 444/600
2090/2090 [==============================] - 0s 235us/step - loss: 0.0294 - accuracy: 0.9914 - val_loss: 10.2315 - val_accuracy: 0.0992
Epoch 445/600
2090/2090 [==============================] - 1s 435us/step - loss: 0.0259 - accuracy: 0.9923 - val_loss: 10.2325 - val_accuracy: 0.0959
Epoch 446/600
2090/2090 [==============================] - 1s 433us/step - loss: 0.0271 - accuracy: 0.9919 - val_loss: 10.2611 - val_accuracy: 0.0959
Epoch 447/600
2090/2090 [==============================] - 0s 212us/step - loss: 0.0287 - accuracy: 

Epoch 550/600
2090/2090 [==============================] - 1s 385us/step - loss: 0.0212 - accuracy: 0.9928 - val_loss: 10.8301 - val_accuracy: 0.0970
Epoch 551/600
2090/2090 [==============================] - 0s 239us/step - loss: 0.0195 - accuracy: 0.9928 - val_loss: 10.7975 - val_accuracy: 0.0959
Epoch 552/600
2090/2090 [==============================] - 1s 482us/step - loss: 0.0193 - accuracy: 0.9947 - val_loss: 10.7684 - val_accuracy: 0.0925
Epoch 553/600
2090/2090 [==============================] - 1s 530us/step - loss: 0.0223 - accuracy: 0.9928 - val_loss: 10.7664 - val_accuracy: 0.0903
Epoch 554/600
2090/2090 [==============================] - 1s 388us/step - loss: 0.0223 - accuracy: 0.9923 - val_loss: 10.7789 - val_accuracy: 0.0858
Epoch 555/600
2090/2090 [==============================] - 1s 535us/step - loss: 0.0191 - accuracy: 0.9904 - val_loss: 10.7982 - val_accuracy: 0.0836
Epoch 556/600
2090/2090 [==============================] - 1s 571us/step - loss: 0.0216 - accuracy: 

In [25]:
y_pred = model.predict(test_vector)
y_pred_label = pd.DataFrame(np.argmax(y_pred, axis=1))
predictions = pd.merge(y_pred_label, y_labels_lut, how='left', left_on=0, right_on='Labels')[category_columns]
accuracy(test_y, predictions)

,Precision,Recall,F1-Score,Support
math,0.48,0.38,0.42,251.0
dp,0.32,0.35,0.34,193.0
implementation,0.36,0.31,0.33,267.0
greedy,0.43,0.27,0.33,248.0
data structures,0.38,0.30,0.33,158.0
sortings,0.27,0.06,0.10,96.0
dfs,0.30,0.15,0.20,93.0
graphs,0.50,0.33,0.40,104.0
trees,0.63,0.42,0.51,92.0
constructive algorithms,0.29,0.11,0.16,133.0


#### RMSProp Optimizer

In [26]:
epochs, batch_size = 600, 128
with K.tf.device('/device:GPU:0'):
    model = gen_model(keras.optimizers.RMSprop(lr=0.001))
    model.fit(train_vector, train_y_onehot,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.3,
              callbacks=[lr_reduction])

Train on 2090 samples, validate on 897 samples
Epoch 1/600
2090/2090 [==============================] - 0s 171us/step - loss: 5.8581 - accuracy: 0.0593 - val_loss: 5.3480 - val_accuracy: 0.0825
Epoch 2/600
2090/2090 [==============================] - 0s 184us/step - loss: 5.1327 - accuracy: 0.0813 - val_loss: 5.2468 - val_accuracy: 0.0836
Epoch 3/600
2090/2090 [==============================] - 1s 260us/step - loss: 5.0104 - accuracy: 0.0861 - val_loss: 5.2295 - val_accuracy: 0.0881
Epoch 4/600
2090/2090 [==============================] - 1s 367us/step - loss: 4.9368 - accuracy: 0.0976 - val_loss: 5.2211 - val_accuracy: 0.0970
Epoch 5/600
2090/2090 [==============================] - 1s 531us/step - loss: 4.8418 - accuracy: 0.0995 - val_loss: 5.2284 - val_accuracy: 0.1014
Epoch 6/600
2090/2090 [==============================] - 2s 721us/step - loss: 4.7652 - accuracy: 0.1124 - val_loss: 5.2285 - val_accuracy: 0.0959
Epoch 7/600
2090/2090 [==============================] - 1s 393us/step 

2090/2090 [==============================] - 0s 236us/step - loss: 0.3069 - accuracy: 0.9469 - val_loss: 8.0693 - val_accuracy: 0.1048
Epoch 112/600
2090/2090 [==============================] - 1s 338us/step - loss: 0.3049 - accuracy: 0.9459 - val_loss: 8.1195 - val_accuracy: 0.1037
Epoch 113/600
2090/2090 [==============================] - 1s 435us/step - loss: 0.2943 - accuracy: 0.9507 - val_loss: 8.1473 - val_accuracy: 0.0981
Epoch 114/600
2090/2090 [==============================] - 1s 387us/step - loss: 0.2756 - accuracy: 0.9569 - val_loss: 8.1891 - val_accuracy: 0.1037
Epoch 115/600
2090/2090 [==============================] - 1s 430us/step - loss: 0.2839 - accuracy: 0.9478 - val_loss: 8.2067 - val_accuracy: 0.1048
Epoch 116/600
2090/2090 [==============================] - 1s 240us/step - loss: 0.2797 - accuracy: 0.9522 - val_loss: 8.1739 - val_accuracy: 0.1048
Epoch 117/600
2090/2090 [==============================] - 0s 239us/step - loss: 0.2813 - accuracy: 0.9522 - val_loss: 8

2090/2090 [==============================] - 1s 432us/step - loss: 0.0981 - accuracy: 0.9828 - val_loss: 9.2002 - val_accuracy: 0.1014
Epoch 167/600
2090/2090 [==============================] - 1s 384us/step - loss: 0.0973 - accuracy: 0.9833 - val_loss: 9.2260 - val_accuracy: 0.0992
Epoch 168/600
2090/2090 [==============================] - 1s 341us/step - loss: 0.0958 - accuracy: 0.9804 - val_loss: 9.2066 - val_accuracy: 0.1037
Epoch 169/600
2090/2090 [==============================] - 0s 188us/step - loss: 0.0951 - accuracy: 0.9823 - val_loss: 9.2344 - val_accuracy: 0.0992
Epoch 170/600
2090/2090 [==============================] - 1s 392us/step - loss: 0.0931 - accuracy: 0.9823 - val_loss: 9.2972 - val_accuracy: 0.1081
Epoch 171/600
2090/2090 [==============================] - 1s 383us/step - loss: 0.0777 - accuracy: 0.9871 - val_loss: 9.2888 - val_accuracy: 0.1048
Epoch 172/600
2090/2090 [==============================] - 0s 191us/step - loss: 0.0845 - accuracy: 0.9852 - val_loss: 9

2090/2090 [==============================] - 1s 241us/step - loss: 0.0447 - accuracy: 0.9914 - val_loss: 10.0901 - val_accuracy: 0.0948
Epoch 222/600
2090/2090 [==============================] - 1s 433us/step - loss: 0.0478 - accuracy: 0.9885 - val_loss: 10.0311 - val_accuracy: 0.1014
Epoch 223/600
2090/2090 [==============================] - 1s 247us/step - loss: 0.0465 - accuracy: 0.9904 - val_loss: 10.0389 - val_accuracy: 0.0981
Epoch 224/600
2090/2090 [==============================] - 1s 524us/step - loss: 0.0446 - accuracy: 0.9909 - val_loss: 10.1268 - val_accuracy: 0.0948
Epoch 225/600
2090/2090 [==============================] - 1s 393us/step - loss: 0.0477 - accuracy: 0.9900 - val_loss: 10.0760 - val_accuracy: 0.1003
Epoch 226/600
2090/2090 [==============================] - 1s 329us/step - loss: 0.0413 - accuracy: 0.9909 - val_loss: 10.1205 - val_accuracy: 0.0981
Epoch 227/600
2090/2090 [==============================] - 1s 389us/step - loss: 0.0428 - accuracy: 0.9895 - val_l

Epoch 276/600
2090/2090 [==============================] - 0s 210us/step - loss: 0.0312 - accuracy: 0.9914 - val_loss: 10.7564 - val_accuracy: 0.0948
Epoch 277/600
2090/2090 [==============================] - 1s 249us/step - loss: 0.0279 - accuracy: 0.9909 - val_loss: 10.7097 - val_accuracy: 0.0981
Epoch 278/600
2090/2090 [==============================] - 0s 121us/step - loss: 0.0272 - accuracy: 0.9923 - val_loss: 10.7731 - val_accuracy: 0.0970
Epoch 279/600
2090/2090 [==============================] - 1s 430us/step - loss: 0.0299 - accuracy: 0.9909 - val_loss: 10.6740 - val_accuracy: 0.1003
Epoch 280/600
2090/2090 [==============================] - 1s 580us/step - loss: 0.0287 - accuracy: 0.9923 - val_loss: 10.7196 - val_accuracy: 0.0959
Epoch 281/600
2090/2090 [==============================] - 1s 384us/step - loss: 0.0276 - accuracy: 0.9904 - val_loss: 10.7301 - val_accuracy: 0.0959
Epoch 282/600
2090/2090 [==============================] - 1s 338us/step - loss: 0.0285 - accuracy: 

Epoch 385/600
2090/2090 [==============================] - 1s 577us/step - loss: 0.0209 - accuracy: 0.9919 - val_loss: 11.6612 - val_accuracy: 0.0970
Epoch 386/600
2090/2090 [==============================] - 1s 336us/step - loss: 0.0198 - accuracy: 0.9933 - val_loss: 11.6235 - val_accuracy: 0.0970
Epoch 387/600
2090/2090 [==============================] - 1s 306us/step - loss: 0.0180 - accuracy: 0.9923 - val_loss: 11.6918 - val_accuracy: 0.0948
Epoch 388/600
2090/2090 [==============================] - 0s 223us/step - loss: 0.0171 - accuracy: 0.9938 - val_loss: 11.7554 - val_accuracy: 0.0959
Epoch 389/600
2090/2090 [==============================] - 1s 580us/step - loss: 0.0161 - accuracy: 0.9938 - val_loss: 11.6994 - val_accuracy: 0.0970
Epoch 390/600
2090/2090 [==============================] - 1s 338us/step - loss: 0.0230 - accuracy: 0.9914 - val_loss: 11.6793 - val_accuracy: 0.0981
Epoch 391/600
2090/2090 [==============================] - 1s 249us/step - loss: 0.0206 - accuracy: 

Epoch 494/600
2090/2090 [==============================] - 0s 209us/step - loss: 0.0186 - accuracy: 0.9928 - val_loss: 12.2844 - val_accuracy: 0.0970
Epoch 495/600
2090/2090 [==============================] - 1s 367us/step - loss: 0.0174 - accuracy: 0.9914 - val_loss: 12.3222 - val_accuracy: 0.0948
Epoch 496/600
2090/2090 [==============================] - 1s 389us/step - loss: 0.0184 - accuracy: 0.9904 - val_loss: 12.2788 - val_accuracy: 0.0936
Epoch 497/600
2090/2090 [==============================] - 1s 240us/step - loss: 0.0176 - accuracy: 0.9909 - val_loss: 12.2420 - val_accuracy: 0.0936
Epoch 498/600
2090/2090 [==============================] - 1s 241us/step - loss: 0.0160 - accuracy: 0.9943 - val_loss: 12.2760 - val_accuracy: 0.0970
Epoch 499/600
2090/2090 [==============================] - 1s 525us/step - loss: 0.0171 - accuracy: 0.9923 - val_loss: 12.2988 - val_accuracy: 0.0970
Epoch 500/600
2090/2090 [==============================] - 1s 340us/step - loss: 0.0199 - accuracy: 

In [28]:

y_pred = model.predict(test_vector)
y_pred_label = pd.DataFrame(np.argmax(y_pred, axis=1))
predictions = pd.merge(y_pred_label, y_labels_lut, how='left', left_on=0, right_on='Labels')[category_columns]
accuracy(test_y, predictions)

,Precision,Recall,F1-Score,Support
math,0.46,0.41,0.43,251.0
dp,0.36,0.40,0.38,193.0
implementation,0.42,0.35,0.38,267.0
greedy,0.45,0.30,0.36,248.0
data structures,0.37,0.23,0.28,158.0
sortings,0.36,0.09,0.15,96.0
dfs,0.34,0.15,0.21,93.0
graphs,0.51,0.36,0.42,104.0
trees,0.65,0.36,0.46,92.0
constructive algorithms,0.35,0.17,0.22,133.0


#### Adagrad

In [39]:
epochs, batch_size = 600, 128
with K.tf.device('/device:GPU:0'):

    model = gen_model(keras.optimizers.Adagrad(lr=0.01))
    model.fit(train_vector, train_y_onehot,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.3,
              callbacks=[lr_reduction])

Train on 2090 samples, validate on 897 samples
Epoch 1/600
2090/2090 [==============================] - 1s 254us/step - loss: 5.4311 - accuracy: 0.0756 - val_loss: 5.2127 - val_accuracy: 0.0970
Epoch 2/600
2090/2090 [==============================] - 0s 135us/step - loss: 4.9577 - accuracy: 0.0938 - val_loss: 5.1531 - val_accuracy: 0.0914
Epoch 3/600
2090/2090 [==============================] - 0s 136us/step - loss: 4.7961 - accuracy: 0.1124 - val_loss: 5.1290 - val_accuracy: 0.0992
Epoch 4/600
2090/2090 [==============================] - 0s 233us/step - loss: 4.6513 - accuracy: 0.1249 - val_loss: 5.0962 - val_accuracy: 0.1093
Epoch 5/600
2090/2090 [==============================] - 1s 432us/step - loss: 4.5306 - accuracy: 0.1455 - val_loss: 5.0903 - val_accuracy: 0.1081
Epoch 6/600
2090/2090 [==============================] - 1s 532us/step - loss: 4.4321 - accuracy: 0.1359 - val_loss: 5.0751 - val_accuracy: 0.1137
Epoch 7/600
2090/2090 [==============================] - 1s 386us/step 

2090/2090 [==============================] - 1s 535us/step - loss: 1.4066 - accuracy: 0.6799 - val_loss: 6.2714 - val_accuracy: 0.1115
Epoch 112/600
2090/2090 [==============================] - 0s 104us/step - loss: 1.3825 - accuracy: 0.6823 - val_loss: 6.2823 - val_accuracy: 0.1093
Epoch 113/600
2090/2090 [==============================] - 0s 192us/step - loss: 1.3997 - accuracy: 0.6775 - val_loss: 6.2878 - val_accuracy: 0.1093
Epoch 114/600
2090/2090 [==============================] - 0s 149us/step - loss: 1.3580 - accuracy: 0.7019 - val_loss: 6.2955 - val_accuracy: 0.1093
Epoch 115/600
2090/2090 [==============================] - 0s 81us/step - loss: 1.3622 - accuracy: 0.7029 - val_loss: 6.3001 - val_accuracy: 0.1104
Epoch 116/600
2090/2090 [==============================] - 1s 479us/step - loss: 1.3606 - accuracy: 0.6923 - val_loss: 6.3065 - val_accuracy: 0.1104
Epoch 117/600
2090/2090 [==============================] - 1s 241us/step - loss: 1.3831 - accuracy: 0.6780 - val_loss: 6.

2090/2090 [==============================] - 0s 142us/step - loss: 0.8448 - accuracy: 0.8129 - val_loss: 6.7756 - val_accuracy: 0.1059
Epoch 222/600
2090/2090 [==============================] - 1s 579us/step - loss: 0.8677 - accuracy: 0.8148 - val_loss: 6.7841 - val_accuracy: 0.1059
Epoch 223/600
2090/2090 [==============================] - 1s 528us/step - loss: 0.8344 - accuracy: 0.8273 - val_loss: 6.7899 - val_accuracy: 0.1037
Epoch 224/600
2090/2090 [==============================] - 1s 384us/step - loss: 0.8422 - accuracy: 0.8211 - val_loss: 6.7921 - val_accuracy: 0.1059
Epoch 225/600
2090/2090 [==============================] - 1s 340us/step - loss: 0.8292 - accuracy: 0.8177 - val_loss: 6.7932 - val_accuracy: 0.1026
Epoch 226/600
2090/2090 [==============================] - 1s 480us/step - loss: 0.8262 - accuracy: 0.8282 - val_loss: 6.7980 - val_accuracy: 0.1081
Epoch 227/600
2090/2090 [==============================] - 1s 385us/step - loss: 0.8405 - accuracy: 0.8182 - val_loss: 6

2090/2090 [==============================] - 1s 279us/step - loss: 0.5938 - accuracy: 0.8809 - val_loss: 7.1035 - val_accuracy: 0.1059
Epoch 332/600
2090/2090 [==============================] - 1s 341us/step - loss: 0.5888 - accuracy: 0.8818 - val_loss: 7.1065 - val_accuracy: 0.1070
Epoch 333/600
2090/2090 [==============================] - 0s 140us/step - loss: 0.6029 - accuracy: 0.8785 - val_loss: 7.1084 - val_accuracy: 0.1070
Epoch 334/600
2090/2090 [==============================] - 1s 390us/step - loss: 0.5941 - accuracy: 0.8794 - val_loss: 7.1108 - val_accuracy: 0.1059
Epoch 335/600
2090/2090 [==============================] - 1s 624us/step - loss: 0.5938 - accuracy: 0.8847 - val_loss: 7.1146 - val_accuracy: 0.1070
Epoch 336/600
2090/2090 [==============================] - 1s 579us/step - loss: 0.5942 - accuracy: 0.8804 - val_loss: 7.1124 - val_accuracy: 0.1037
Epoch 337/600
2090/2090 [==============================] - 0s 193us/step - loss: 0.6014 - accuracy: 0.8699 - val_loss: 7

2090/2090 [==============================] - 1s 362us/step - loss: 0.4614 - accuracy: 0.9024 - val_loss: 7.3335 - val_accuracy: 0.1037
Epoch 442/600
2090/2090 [==============================] - 0s 85us/step - loss: 0.4671 - accuracy: 0.9091 - val_loss: 7.3366 - val_accuracy: 0.1059
Epoch 443/600
2090/2090 [==============================] - 0s 208us/step - loss: 0.4526 - accuracy: 0.9187 - val_loss: 7.3485 - val_accuracy: 0.1059
Epoch 444/600
2090/2090 [==============================] - 0s 190us/step - loss: 0.4647 - accuracy: 0.9129 - val_loss: 7.3427 - val_accuracy: 0.1048
Epoch 445/600
2090/2090 [==============================] - 1s 483us/step - loss: 0.4628 - accuracy: 0.9158 - val_loss: 7.3461 - val_accuracy: 0.1048
Epoch 446/600
2090/2090 [==============================] - 1s 432us/step - loss: 0.4576 - accuracy: 0.9144 - val_loss: 7.3463 - val_accuracy: 0.1059
Epoch 447/600
2090/2090 [==============================] - 1s 433us/step - loss: 0.4787 - accuracy: 0.8971 - val_loss: 7.

2090/2090 [==============================] - 1s 383us/step - loss: 0.4088 - accuracy: 0.9239 - val_loss: 7.4439 - val_accuracy: 0.1048
Epoch 497/600
2090/2090 [==============================] - 1s 391us/step - loss: 0.4135 - accuracy: 0.9211 - val_loss: 7.4462 - val_accuracy: 0.1037
Epoch 498/600
2090/2090 [==============================] - 1s 284us/step - loss: 0.4127 - accuracy: 0.9177 - val_loss: 7.4489 - val_accuracy: 0.1037
Epoch 499/600
2090/2090 [==============================] - 1s 289us/step - loss: 0.4341 - accuracy: 0.9134 - val_loss: 7.4461 - val_accuracy: 0.1048
Epoch 500/600
2090/2090 [==============================] - 1s 390us/step - loss: 0.4087 - accuracy: 0.9172 - val_loss: 7.4445 - val_accuracy: 0.1048
Epoch 501/600
2090/2090 [==============================] - 1s 572us/step - loss: 0.4244 - accuracy: 0.9115 - val_loss: 7.4469 - val_accuracy: 0.1026
Epoch 502/600
2090/2090 [==============================] - 1s 292us/step - loss: 0.3995 - accuracy: 0.9325 - val_loss: 7

2090/2090 [==============================] - 1s 335us/step - loss: 0.3790 - accuracy: 0.9325 - val_loss: 7.5268 - val_accuracy: 0.1048
Epoch 552/600
2090/2090 [==============================] - 1s 298us/step - loss: 0.3612 - accuracy: 0.9411 - val_loss: 7.5318 - val_accuracy: 0.1048
Epoch 553/600
2090/2090 [==============================] - 1s 374us/step - loss: 0.3665 - accuracy: 0.9368 - val_loss: 7.5330 - val_accuracy: 0.1048
Epoch 554/600
2090/2090 [==============================] - 1s 337us/step - loss: 0.3677 - accuracy: 0.9397 - val_loss: 7.5305 - val_accuracy: 0.1037
Epoch 555/600
2090/2090 [==============================] - 1s 243us/step - loss: 0.3761 - accuracy: 0.9335 - val_loss: 7.5303 - val_accuracy: 0.1037
Epoch 556/600
2090/2090 [==============================] - 1s 348us/step - loss: 0.3858 - accuracy: 0.9330 - val_loss: 7.5337 - val_accuracy: 0.1026
Epoch 557/600
2090/2090 [==============================] - 0s 179us/step - loss: 0.3731 - accuracy: 0.9340 - val_loss: 7

In [43]:
y_pred = model.predict(test_vector)
y_pred_label = pd.DataFrame(np.argmax(y_pred, axis=1))
predictions = pd.merge(y_pred_label, y_labels_lut, how='left', left_on=0, right_on='Labels')[category_columns]
accuracy(test_y, predictions)

,Precision,Recall,F1-Score,Support
math,0.00,0.00,0.00,251.0
dp,0.00,0.00,0.00,193.0
implementation,0.00,0.00,0.00,267.0
greedy,0.00,0.00,0.00,248.0
data structures,0.00,0.00,0.00,158.0
sortings,0.00,0.00,0.00,96.0
dfs,0.00,0.00,0.00,93.0
graphs,0.00,0.00,0.00,104.0
trees,0.00,0.00,0.00,92.0
constructive algorithms,0.17,0.05,0.07,133.0


#### Adadelta

In [31]:
epochs, batch_size = 600, 128
with K.tf.device('/device:GPU:0'):

    model = gen_model(keras.optimizers.Adadelta(lr=1.0))
    model.fit(train_vector, train_y_onehot,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.3,
              callbacks=[lr_reduction])

Train on 2090 samples, validate on 897 samples
Epoch 1/600
2090/2090 [==============================] - 1s 340us/step - loss: 6.2478 - accuracy: 0.0407 - val_loss: 6.1751 - val_accuracy: 0.0792
Epoch 2/600
2090/2090 [==============================] - 1s 241us/step - loss: 6.0154 - accuracy: 0.0746 - val_loss: 5.7911 - val_accuracy: 0.0825
Epoch 3/600
2090/2090 [==============================] - 1s 533us/step - loss: 5.5214 - accuracy: 0.0766 - val_loss: 5.4256 - val_accuracy: 0.0825
Epoch 4/600
2090/2090 [==============================] - 0s 164us/step - loss: 5.2668 - accuracy: 0.0789 - val_loss: 5.3125 - val_accuracy: 0.0825
Epoch 5/600
2090/2090 [==============================] - 1s 313us/step - loss: 5.1812 - accuracy: 0.0761 - val_loss: 5.2653 - val_accuracy: 0.0825
Epoch 6/600
2090/2090 [==============================] - 1s 336us/step - loss: 5.1230 - accuracy: 0.0761 - val_loss: 5.2488 - val_accuracy: 0.0836
Epoch 7/600
2090/2090 [==============================] - 1s 342us/step 

2090/2090 [==============================] - 1s 333us/step - loss: 2.6871 - accuracy: 0.3699 - val_loss: 5.1372 - val_accuracy: 0.0981
Epoch 112/600
2090/2090 [==============================] - 1s 382us/step - loss: 2.6764 - accuracy: 0.3746 - val_loss: 5.1514 - val_accuracy: 0.1026
Epoch 113/600
2090/2090 [==============================] - 1s 291us/step - loss: 2.6546 - accuracy: 0.3761 - val_loss: 5.1515 - val_accuracy: 0.1014
Epoch 114/600
2090/2090 [==============================] - 1s 243us/step - loss: 2.6285 - accuracy: 0.3804 - val_loss: 5.1486 - val_accuracy: 0.1115
Epoch 115/600
2090/2090 [==============================] - 0s 236us/step - loss: 2.6153 - accuracy: 0.3852 - val_loss: 5.1657 - val_accuracy: 0.1104
Epoch 116/600
2090/2090 [==============================] - 1s 385us/step - loss: 2.5903 - accuracy: 0.3981 - val_loss: 5.1809 - val_accuracy: 0.0992
Epoch 117/600
2090/2090 [==============================] - 1s 479us/step - loss: 2.5532 - accuracy: 0.4043 - val_loss: 5

Epoch 166/600
2090/2090 [==============================] - 1s 284us/step - loss: 1.6079 - accuracy: 0.5971 - val_loss: 5.5582 - val_accuracy: 0.1059
Epoch 167/600
2090/2090 [==============================] - 1s 337us/step - loss: 1.5629 - accuracy: 0.6043 - val_loss: 5.5766 - val_accuracy: 0.0992
Epoch 168/600
2090/2090 [==============================] - 1s 584us/step - loss: 1.5428 - accuracy: 0.6244 - val_loss: 5.5786 - val_accuracy: 0.1059
Epoch 169/600
2090/2090 [==============================] - 1s 429us/step - loss: 1.5286 - accuracy: 0.6172 - val_loss: 5.5885 - val_accuracy: 0.1003
Epoch 170/600
2090/2090 [==============================] - 1s 530us/step - loss: 1.5143 - accuracy: 0.6340 - val_loss: 5.5987 - val_accuracy: 0.0948
Epoch 171/600
2090/2090 [==============================] - 1s 395us/step - loss: 1.5198 - accuracy: 0.6268 - val_loss: 5.6007 - val_accuracy: 0.1037
Epoch 172/600
2090/2090 [==============================] - 1s 423us/step - loss: 1.4740 - accuracy: 0.6445

Epoch 221/600
2090/2090 [==============================] - 1s 432us/step - loss: 0.8467 - accuracy: 0.8230 - val_loss: 6.0190 - val_accuracy: 0.0981
Epoch 222/600
2090/2090 [==============================] - 1s 352us/step - loss: 0.8652 - accuracy: 0.8091 - val_loss: 6.0271 - val_accuracy: 0.1026
Epoch 223/600
2090/2090 [==============================] - 0s 185us/step - loss: 0.8308 - accuracy: 0.8378 - val_loss: 6.0452 - val_accuracy: 0.1037
Epoch 224/600
2090/2090 [==============================] - 1s 376us/step - loss: 0.8217 - accuracy: 0.8278 - val_loss: 6.0660 - val_accuracy: 0.1070
Epoch 225/600
2090/2090 [==============================] - 1s 483us/step - loss: 0.8078 - accuracy: 0.8378 - val_loss: 6.0540 - val_accuracy: 0.0959
Epoch 226/600
2090/2090 [==============================] - 0s 66us/step - loss: 0.8079 - accuracy: 0.8378 - val_loss: 6.0602 - val_accuracy: 0.0959
Epoch 227/600
2090/2090 [==============================] - 0s 169us/step - loss: 0.8150 - accuracy: 0.8258 

2090/2090 [==============================] - 1s 577us/step - loss: 0.5058 - accuracy: 0.9019 - val_loss: 6.4092 - val_accuracy: 0.1037
Epoch 277/600
2090/2090 [==============================] - 1s 289us/step - loss: 0.4867 - accuracy: 0.9096 - val_loss: 6.4014 - val_accuracy: 0.0970
Epoch 278/600
2090/2090 [==============================] - 1s 530us/step - loss: 0.5088 - accuracy: 0.8990 - val_loss: 6.4076 - val_accuracy: 0.0992
Epoch 279/600
2090/2090 [==============================] - 1s 486us/step - loss: 0.4879 - accuracy: 0.9057 - val_loss: 6.4280 - val_accuracy: 0.1003
Epoch 280/600
2090/2090 [==============================] - 0s 175us/step - loss: 0.4816 - accuracy: 0.9129 - val_loss: 6.4221 - val_accuracy: 0.0970
Epoch 281/600
2090/2090 [==============================] - 0s 163us/step - loss: 0.4901 - accuracy: 0.9053 - val_loss: 6.4409 - val_accuracy: 0.0948
Epoch 282/600
2090/2090 [==============================] - 0s 91us/step - loss: 0.4869 - accuracy: 0.8986 - val_loss: 6.

2090/2090 [==============================] - 1s 474us/step - loss: 0.3290 - accuracy: 0.9450 - val_loss: 6.7313 - val_accuracy: 0.0992
Epoch 332/600
2090/2090 [==============================] - 1s 352us/step - loss: 0.3235 - accuracy: 0.9435 - val_loss: 6.7287 - val_accuracy: 0.0981
Epoch 333/600
2090/2090 [==============================] - 1s 372us/step - loss: 0.3277 - accuracy: 0.9431 - val_loss: 6.7393 - val_accuracy: 0.0959
Epoch 334/600
2090/2090 [==============================] - 1s 480us/step - loss: 0.3257 - accuracy: 0.9478 - val_loss: 6.7302 - val_accuracy: 0.0981
Epoch 335/600
2090/2090 [==============================] - 1s 442us/step - loss: 0.3079 - accuracy: 0.9488 - val_loss: 6.7344 - val_accuracy: 0.1003
Epoch 336/600
2090/2090 [==============================] - 1s 570us/step - loss: 0.3193 - accuracy: 0.9464 - val_loss: 6.7695 - val_accuracy: 0.1037
Epoch 337/600
2090/2090 [==============================] - 1s 258us/step - loss: 0.3050 - accuracy: 0.9526 - val_loss: 6

2090/2090 [==============================] - 1s 434us/step - loss: 0.1840 - accuracy: 0.9660 - val_loss: 7.1938 - val_accuracy: 0.0881
Epoch 442/600
2090/2090 [==============================] - 0s 143us/step - loss: 0.1744 - accuracy: 0.9746 - val_loss: 7.2137 - val_accuracy: 0.0903
Epoch 443/600
2090/2090 [==============================] - 1s 481us/step - loss: 0.1740 - accuracy: 0.9679 - val_loss: 7.2059 - val_accuracy: 0.0903
Epoch 444/600
2090/2090 [==============================] - 2s 723us/step - loss: 0.1711 - accuracy: 0.9718 - val_loss: 7.2146 - val_accuracy: 0.0870
Epoch 445/600
2090/2090 [==============================] - 1s 529us/step - loss: 0.1614 - accuracy: 0.9718 - val_loss: 7.2177 - val_accuracy: 0.0892
Epoch 446/600
2090/2090 [==============================] - 1s 627us/step - loss: 0.1749 - accuracy: 0.9694 - val_loss: 7.2174 - val_accuracy: 0.0892
Epoch 447/600
2090/2090 [==============================] - 1s 387us/step - loss: 0.1626 - accuracy: 0.9746 - val_loss: 7

2090/2090 [==============================] - 1s 528us/step - loss: 0.1433 - accuracy: 0.9761 - val_loss: 7.3595 - val_accuracy: 0.0881
Epoch 497/600
2090/2090 [==============================] - 1s 260us/step - loss: 0.1326 - accuracy: 0.9813 - val_loss: 7.3764 - val_accuracy: 0.0858
Epoch 498/600
2090/2090 [==============================] - 1s 285us/step - loss: 0.1401 - accuracy: 0.9799 - val_loss: 7.3835 - val_accuracy: 0.0870
Epoch 499/600
2090/2090 [==============================] - 0s 174us/step - loss: 0.1302 - accuracy: 0.9818 - val_loss: 7.3954 - val_accuracy: 0.0903
Epoch 500/600
2090/2090 [==============================] - 1s 335us/step - loss: 0.1331 - accuracy: 0.9761 - val_loss: 7.4089 - val_accuracy: 0.0870
Epoch 501/600
2090/2090 [==============================] - 1s 537us/step - loss: 0.1324 - accuracy: 0.9785 - val_loss: 7.4078 - val_accuracy: 0.0858
Epoch 502/600
2090/2090 [==============================] - 1s 381us/step - loss: 0.1233 - accuracy: 0.9823 - val_loss: 7

In [33]:

y_pred = model.predict(test_vector)
y_pred_label = pd.DataFrame(np.argmax(y_pred, axis=1))
predictions = pd.merge(y_pred_label, y_labels_lut, how='left', left_on=0, right_on='Labels')[category_columns]
accuracy(test_y, predictions)

,Precision,Recall,F1-Score,Support
math,0.47,0.43,0.45,251.0
dp,0.34,0.35,0.35,193.0
implementation,0.40,0.34,0.37,267.0
greedy,0.45,0.32,0.37,248.0
data structures,0.33,0.26,0.29,158.0
sortings,0.17,0.07,0.10,96.0
dfs,0.33,0.23,0.27,93.0
graphs,0.49,0.40,0.44,104.0
trees,0.66,0.52,0.58,92.0
constructive algorithms,0.25,0.14,0.18,133.0


### Binary Relevance

In [35]:
def gen_model_genre(optimizer):
    with K.tf.device('/device:GPU:0'):
        model = Sequential()
        model.add(Dense(800, activation='relu', input_shape=(512,)))
        model.add(Dropout(0.25))
        model.add(Dense(15, activation='sigmoid'))
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model
lr_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)


In [36]:
prob_thresh = (train_y.sum()/train_y.shape[0]).clip(upper=0.5)
prob_thresh

math                       0.247071
dp                         0.216940
implementation             0.318045
greedy                     0.225979
data structures            0.178105
sortings                   0.095079
dfs                        0.095079
graphs                     0.110144
trees                      0.085705
constructive algorithms    0.132240
brute force                0.154670
strings                    0.077335
binary search              0.100770
number theory              0.068966
geometry                   0.044526
dtype: float64

In [41]:
epochs, batch_size = 1000, 128
with K.tf.device('/device:GPU:0'):
    model = gen_model_genre(keras.optimizers.Adam(lr=0.001))
    model.fit(train_vector, train_y,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_split=0.3,
              callbacks=[lr_reduction])

Train on 2090 samples, validate on 897 samples
Epoch 1/1000
2090/2090 [==============================] - 0s 114us/step - loss: 5.6574 - accuracy: 0.2368 - val_loss: 5.5027 - val_accuracy: 0.2653
Epoch 2/1000
2090/2090 [==============================] - 0s 45us/step - loss: 5.3849 - accuracy: 0.2368 - val_loss: 5.3407 - val_accuracy: 0.2475
Epoch 3/1000
2090/2090 [==============================] - 0s 44us/step - loss: 5.2027 - accuracy: 0.2273 - val_loss: 5.1759 - val_accuracy: 0.2475
Epoch 4/1000
2090/2090 [==============================] - 0s 40us/step - loss: 5.0214 - accuracy: 0.2301 - val_loss: 5.0423 - val_accuracy: 0.2843
Epoch 5/1000
2090/2090 [==============================] - 0s 42us/step - loss: 4.8473 - accuracy: 0.3124 - val_loss: 4.9304 - val_accuracy: 0.3255
Epoch 6/1000
2090/2090 [==============================] - 0s 43us/step - loss: 4.7194 - accuracy: 0.3349 - val_loss: 4.9003 - val_accuracy: 0.3166
Epoch 7/1000
2090/2090 [==============================] - 0s 43us/step

2090/2090 [==============================] - 0s 42us/step - loss: 3.1124 - accuracy: 0.5287 - val_loss: 5.5805 - val_accuracy: 0.2375
Epoch 57/1000
2090/2090 [==============================] - 0s 42us/step - loss: 3.0778 - accuracy: 0.5397 - val_loss: 5.5890 - val_accuracy: 0.2397
Epoch 58/1000
2090/2090 [==============================] - 0s 42us/step - loss: 3.0644 - accuracy: 0.5407 - val_loss: 5.6202 - val_accuracy: 0.2497
Epoch 59/1000
2090/2090 [==============================] - 0s 43us/step - loss: 3.0496 - accuracy: 0.5488 - val_loss: 5.6384 - val_accuracy: 0.2419
Epoch 60/1000
2090/2090 [==============================] - 0s 43us/step - loss: 3.0213 - accuracy: 0.5488 - val_loss: 5.6460 - val_accuracy: 0.2497
Epoch 61/1000
2090/2090 [==============================] - 0s 45us/step - loss: 2.9892 - accuracy: 0.5397 - val_loss: 5.6652 - val_accuracy: 0.2453
Epoch 62/1000
2090/2090 [==============================] - 0s 44us/step - loss: 2.9820 - accuracy: 0.5517 - val_loss: 5.6902 -

2090/2090 [==============================] - 0s 43us/step - loss: 2.2399 - accuracy: 0.6019 - val_loss: 6.8988 - val_accuracy: 0.2386
Epoch 167/1000
2090/2090 [==============================] - 0s 43us/step - loss: 2.2491 - accuracy: 0.5933 - val_loss: 6.9205 - val_accuracy: 0.2419
Epoch 168/1000
2090/2090 [==============================] - 0s 42us/step - loss: 2.2352 - accuracy: 0.5995 - val_loss: 6.8715 - val_accuracy: 0.2297
Epoch 169/1000
2090/2090 [==============================] - 0s 44us/step - loss: 2.2364 - accuracy: 0.5852 - val_loss: 6.9076 - val_accuracy: 0.2308
Epoch 170/1000
2090/2090 [==============================] - 0s 40us/step - loss: 2.2267 - accuracy: 0.5914 - val_loss: 6.9169 - val_accuracy: 0.2319
Epoch 171/1000
2090/2090 [==============================] - 0s 47us/step - loss: 2.2335 - accuracy: 0.5909 - val_loss: 6.9031 - val_accuracy: 0.2352
Epoch 172/1000
2090/2090 [==============================] - 0s 40us/step - loss: 2.2302 - accuracy: 0.5866 - val_loss: 6.

2090/2090 [==============================] - 0s 42us/step - loss: 2.1539 - accuracy: 0.5923 - val_loss: 7.1970 - val_accuracy: 0.2241
Epoch 222/1000
2090/2090 [==============================] - 0s 42us/step - loss: 2.1476 - accuracy: 0.6010 - val_loss: 7.2272 - val_accuracy: 0.2319
Epoch 223/1000
2090/2090 [==============================] - 0s 49us/step - loss: 2.1493 - accuracy: 0.5971 - val_loss: 7.2134 - val_accuracy: 0.2308
Epoch 224/1000
2090/2090 [==============================] - 0s 43us/step - loss: 2.1489 - accuracy: 0.5943 - val_loss: 7.2019 - val_accuracy: 0.2352
Epoch 225/1000
2090/2090 [==============================] - 0s 43us/step - loss: 2.1506 - accuracy: 0.5837 - val_loss: 7.2283 - val_accuracy: 0.2330
Epoch 226/1000
2090/2090 [==============================] - 0s 43us/step - loss: 2.1432 - accuracy: 0.5933 - val_loss: 7.2434 - val_accuracy: 0.2375
Epoch 227/1000
2090/2090 [==============================] - 0s 43us/step - loss: 2.1527 - accuracy: 0.6057 - val_loss: 7.

2090/2090 [==============================] - 0s 44us/step - loss: 2.0870 - accuracy: 0.6005 - val_loss: 7.5720 - val_accuracy: 0.2297
Epoch 277/1000
2090/2090 [==============================] - 0s 43us/step - loss: 2.0903 - accuracy: 0.6024 - val_loss: 7.5753 - val_accuracy: 0.2241
Epoch 278/1000
2090/2090 [==============================] - 0s 66us/step - loss: 2.0903 - accuracy: 0.6010 - val_loss: 7.6075 - val_accuracy: 0.2263
Epoch 279/1000
2090/2090 [==============================] - 0s 51us/step - loss: 2.0810 - accuracy: 0.5885 - val_loss: 7.6220 - val_accuracy: 0.2308
Epoch 280/1000
2090/2090 [==============================] - 0s 41us/step - loss: 2.0868 - accuracy: 0.5828 - val_loss: 7.6124 - val_accuracy: 0.2319
Epoch 281/1000
2090/2090 [==============================] - 0s 41us/step - loss: 2.0807 - accuracy: 0.5919 - val_loss: 7.6007 - val_accuracy: 0.2241
Epoch 282/1000
2090/2090 [==============================] - 0s 44us/step - loss: 2.0806 - accuracy: 0.5871 - val_loss: 7.

...

Epoch 550/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9299 - accuracy: 0.6278 - val_loss: 10.5865 - val_accuracy: 0.2408
Epoch 551/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9302 - accuracy: 0.6244 - val_loss: 10.6646 - val_accuracy: 0.2397
Epoch 552/1000
2090/2090 [==============================] - 0s 47us/step - loss: 1.9303 - accuracy: 0.5981 - val_loss: 10.6196 - val_accuracy: 0.2375
Epoch 553/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9304 - accuracy: 0.5957 - val_loss: 10.6437 - val_accuracy: 0.2375
Epoch 554/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9281 - accuracy: 0.6431 - val_loss: 10.7248 - val_accuracy: 0.2308
Epoch 555/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9294 - accuracy: 0.6029 - val_loss: 10.7298 - val_accuracy: 0.2319
Epoch 556/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9283 - accuracy: 

Epoch 659/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9176 - accuracy: 0.6086 - val_loss: 11.7676 - val_accuracy: 0.2386
Epoch 660/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9180 - accuracy: 0.6148 - val_loss: 11.8300 - val_accuracy: 0.2352
Epoch 661/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9180 - accuracy: 0.6057 - val_loss: 11.8652 - val_accuracy: 0.2386
Epoch 662/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9167 - accuracy: 0.5852 - val_loss: 11.9119 - val_accuracy: 0.2453
Epoch 663/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9169 - accuracy: 0.6282 - val_loss: 11.9536 - val_accuracy: 0.2363
Epoch 664/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9178 - accuracy: 0.6464 - val_loss: 11.8837 - val_accuracy: 0.2375
Epoch 665/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9190 - accuracy: 

Epoch 768/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9132 - accuracy: 0.6435 - val_loss: 13.0245 - val_accuracy: 0.2386
Epoch 769/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9139 - accuracy: 0.6096 - val_loss: 12.8572 - val_accuracy: 0.2520
Epoch 770/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9135 - accuracy: 0.6397 - val_loss: 12.9147 - val_accuracy: 0.2464
Epoch 771/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9129 - accuracy: 0.6311 - val_loss: 12.9316 - val_accuracy: 0.2475
Epoch 772/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9124 - accuracy: 0.6196 - val_loss: 13.0693 - val_accuracy: 0.2430
Epoch 773/1000
2090/2090 [==============================] - 0s 45us/step - loss: 1.9138 - accuracy: 0.6163 - val_loss: 12.8754 - val_accuracy: 0.2508
Epoch 774/1000
2090/2090 [==============================] - 0s 46us/step - loss: 1.9131 - accuracy: 

Epoch 877/1000
2090/2090 [==============================] - 0s 66us/step - loss: 1.9102 - accuracy: 0.6368 - val_loss: 13.8465 - val_accuracy: 0.2453
Epoch 878/1000
2090/2090 [==============================] - 0s 47us/step - loss: 1.9102 - accuracy: 0.6225 - val_loss: 13.7989 - val_accuracy: 0.2497
Epoch 879/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9108 - accuracy: 0.6325 - val_loss: 13.7142 - val_accuracy: 0.2441
Epoch 880/1000
2090/2090 [==============================] - 0s 43us/step - loss: 1.9117 - accuracy: 0.6512 - val_loss: 13.8206 - val_accuracy: 0.2419
Epoch 881/1000
2090/2090 [==============================] - 0s 46us/step - loss: 1.9102 - accuracy: 0.6215 - val_loss: 13.8676 - val_accuracy: 0.2441
Epoch 882/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9104 - accuracy: 0.6464 - val_loss: 13.7792 - val_accuracy: 0.2497
Epoch 883/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9108 - accuracy: 

Epoch 986/1000
2090/2090 [==============================] - 0s 42us/step - loss: 1.9101 - accuracy: 0.6215 - val_loss: 14.5400 - val_accuracy: 0.2453
Epoch 987/1000
2090/2090 [==============================] - 0s 42us/step - loss: 1.9101 - accuracy: 0.6632 - val_loss: 14.3176 - val_accuracy: 0.2475
Epoch 988/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9099 - accuracy: 0.6483 - val_loss: 14.4830 - val_accuracy: 0.2453
Epoch 989/1000
2090/2090 [==============================] - 0s 43us/step - loss: 1.9103 - accuracy: 0.6483 - val_loss: 14.4241 - val_accuracy: 0.2386
Epoch 990/1000
2090/2090 [==============================] - 0s 40us/step - loss: 1.9102 - accuracy: 0.6478 - val_loss: 14.4649 - val_accuracy: 0.2386
Epoch 991/1000
2090/2090 [==============================] - 0s 42us/step - loss: 1.9098 - accuracy: 0.6435 - val_loss: 14.5164 - val_accuracy: 0.2397
Epoch 992/1000
2090/2090 [==============================] - 0s 44us/step - loss: 1.9096 - accuracy: 

In [42]:
y_pred = model.predict(test_vector)
predictions = pd.DataFrame(index=test_y.index, columns=test_y.columns)
for i in range(y_pred.shape[0]):
    predictions.iloc[i,:] = (y_pred[i,:]>prob_thresh).map({True:1, False:0})
accuracy(test_y, predictions)

,Precision,Recall,F1-Score,Support
math,0.52,0.29,0.37,251.0
dp,0.41,0.26,0.32,193.0
implementation,0.44,0.20,0.28,267.0
greedy,0.54,0.26,0.35,248.0
data structures,0.49,0.31,0.38,158.0
sortings,0.25,0.12,0.17,96.0
dfs,0.37,0.14,0.20,93.0
graphs,0.54,0.37,0.43,104.0
trees,0.80,0.52,0.63,92.0
constructive algorithms,0.35,0.22,0.27,133.0
